In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pandas as pd
import pymeshfix
from scipy import spatial
from tqdm.auto import tqdm

from utils import config_rcparams, set_axes_equal, set_3d_params

In [ ]:
config_rcparams()

In [ ]:
%config InlineBackend.figure_format = 'retina'

# Extraction of the points on the boundary of the point cloud

Let’s assume we have a set of points, $\mathbb{X} = \{\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n\}$, where $\mathbf{x}_i = (x_i, y_i, z_i)$ with $1 \leq i \leq n$, sampling a compact region $\Omega \subset \mathbb{R}^3$. We want to identify the subset of points that lie on the boundary surface $S = \partial \Omega$, called *surface points*.

<div style="text-align:center">
    <img style="margin:20px; width:450px;" src="figures/pc-surf.svg">
</div>

The following steps should be applied to each point, $\mathbf{x}_i$, in $\mathbb{X}$.

<div style="text-align:center">
    <img style="margin:20px; width:750px;" src="figures/pc-surf-extract.svg">
</div>

The following is the simple implementation in Python by using only the `SciPy.spatial` module.

In [ ]:
def extract_surface_points(points, radius):
    """Return surface points on the point cloud.

    Parameters
    ----------
    points : numpy.ndarray
        Point cloud
    radius : number
        The radius of points to create a local neighbourhood
        
    Return
    ------
    numpy.ndarray
        Surface points
    """
    surface_points = []
    tree = spatial.KDTree(points)
    for point in tqdm(points):
        # step 1: extract a local neighbourhood around the query point
        idx = tree.query_ball_point(point, r=radius, eps=0, p=2)
        nbh = points[idx]
    
        # step 2: estimate normal direction at the query point
        X = nbh.copy()
        X = X - np.mean(X, axis=0)
        C = X.T @ X
        U, S, _ = np.linalg.svd(C)
        n =  U[:, np.argmin(S)]
            
        # step 3: search two circular patches within neighbourhood
        centers = [point+n*radius/2,
                   point-n*radius/2]
        for center in centers:
            ii = tree.query_ball_point(center, r=radius/2, eps=0, p=2)
            if len(ii) in [0, 1]:
                surface_points.append(point)
    return np.unique(surface_points, axis=0)

Example:

In [ ]:
# load the point cloud
fname = os.path.join('data', 'PSSACZ.ply')
pcd = o3d.io.read_point_cloud(fname)
points = np.asarray(pcd.points)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax = set_3d_params(ax)
ax.scatter(*points.T, fc='w', ec='k', s=15, lw=0.5)
ax.view_init(25, -70);

In [ ]:
# extract the surface
fname = os.path.join('data', 'PSSACZ_surf.ply')
if os.path.isfile(fname):
    pcd_surf = o3d.io.read_point_cloud(fname)
    surface_points = np.asarray(pcd_surf.points)
else:
    surface_points = extract_surface_points(points, radius=0.3)
    pcd_surf = o3d.geometry.PointCloud(
        o3d.utility.Vector3dVector(surface_points)
    )
    _ = o3d.io.write_point_cloud(fname, pcd_surf)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax = set_3d_params(ax)
ax.scatter(*surface_points.T, fc='w', ec='k', s=15, lw=0.5)
ax.view_init(25, -70);

In [ ]:
def normalize(points):
    """Return point cloud fixed into the unit cube.

    Parameters
    ----------
    points : numpy.ndarray
        Point cloud

    Return
    ------
    numpy.ndarray
        Scaled point cloud
    """
    centroid = np.mean(points, axis=0)
    points -= centroid
    max_dist = np.linalg.norm(points, axis=1).max()
    points /= max_dist
    return points

In [ ]:
# scale the point cloud
fname = os.path.join('data', 'PSSACZ_surf_scaled.ply')
if os.path.isfile(fname):
    pcd_surf_scaled = o3d.io.read_point_cloud(fname)
    surface_points_scaled = np.asarray(pcd_surf_scaled.points)
    normals = np.asarray(pcd_surf_scaled.normals)
else:
    surface_points_scaled = normalize(surface_points)
    pcd_surf_scaled = o3d.geometry.PointCloud(
        o3d.utility.Vector3dVector(surface_points_scaled)
    )
    pcd_surf_scaled = pcd_surf_scaled.voxel_down_sample(voxel_size=0.01)
    pcd_surf_scaled.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=10))
    pcd_surf_scaled.orient_normals_consistent_tangent_plane(k=20)
    normals = np.asarray(pcd_surf_scaled.normals)
    colors = np.c_[np.round(0.5*normals[:, 0]+0.5, 12),
                   np.round(0.5*normals[:, 1]+0.5, 12),
                   np.round(0.5*normals[:, 2]+0.5, 12)]
    pcd_surf_scaled.colors = o3d.utility.Vector3dVector(colors)
    _ = o3d.io.write_point_cloud(fname, pcd_surf_scaled)

In [ ]:
o3d.visualization.draw_plotly([pcd_surf_scaled])

# Surface reconstruction

## Ball pivoting

In [ ]:
distances = pcd_surf_scaled.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd_surf_scaled, o3d.utility.DoubleVector([radius, radius*2]))

In [ ]:
o3d.visualization.draw_plotly([bpa_mesh])

## Poisson

In [ ]:
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd_surf_scaled, depth=8, scale=1.1)

In [ ]:
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = poisson_mesh.vertices
density_mesh.triangles = poisson_mesh.triangles
density_mesh.triangle_normals = poisson_mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)

In [ ]:
o3d.visualization.draw_geometries([density_mesh])

In [ ]:
mask = densities < np.quantile(densities, 0.2)
density_mesh.remove_vertices_by_mask(mask)

In [ ]:
o3d.visualization.draw_plotly([density_mesh])

## WIP. Signed-distance function interpolation

In [ ]:
import polatory
from matplotlib import cm
from skimage import measure
from scipy import interpolate

In [ ]:
points = np.asarray(pcd_surf_scaled.points)
normals = np.asarray(pcd_surf_scaled.normals)
mask = (np.isclose(normals[:, 0], 0, atol=1e-4)
        & np.isclose(normals[:, 1], 0, atol=1e-4))

In [ ]:
_cloud = o3d.geometry.PointCloud(
    o3d.utility.Vector3dVector(points[~mask])
)

In [ ]:
o3d.visualization.draw_geometries([_cloud])

In [ ]:
sdf = polatory.SdfDataGenerator(points[~mask],
                                normals[~mask],
                                5e-4, 5e-3)
sdf_points, sdf_values = sdf.sdf_points, sdf.sdf_values

In [ ]:
sdf_points = np.append(sdf_points, points[mask], axis=0)
sdf_values = np.append(sdf_values, np.zeros((points[mask].shape[0], )))

In [ ]:
_range = (0, 1)
scaler = (sdf_values - sdf_values.min()) / (sdf_values.max() - sdf_values.min())
sdf_values_scaled = scaler * (_range[1] - _range[0]) + _range[0]
colors = cm.viridis(sdf_values_scaled)[..., :-1]

In [ ]:
sdf_cloud = o3d.geometry.PointCloud(
    o3d.utility.Vector3dVector(sdf_points.copy())
)
sdf_cloud.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
o3d.visualization.draw_geometries([sdf_cloud])

In [ ]:
interp = interpolate.RBFInterpolator(sdf_points,
                                     sdf_values,
                                     neighbors=1000,
                                     kernel='linear',  # biharmonic kernel
                                     degree=1)

In [ ]:
xmin, ymin, zmin = np.min(points, axis=0)
xmax, ymax, zmax = np.max(points, axis=0)

In [ ]:
x_ = np.linspace(xmin, xmax, 21)
y_ = np.linspace(ymin, ymax, 21)
z_ = np.linspace(zmin, zmax, 21)
X, Y, Z = np.meshgrid(x_, y_, z_)
grid = np.c_[X.ravel(), Y.ravel(), Z.ravel()]

In [ ]:
_cloud = o3d.geometry.PointCloud(
    o3d.utility.Vector3dVector(grid)
)
o3d.visualization.draw_geometries([pcd_surf_scaled, _cloud])

In [ ]:
grid_val = interp(grid)

In [ ]:
verts, faces, normals, values = measure.marching_cubes(
    grid_val.reshape(X.shape), 0)

In [ ]:
ex = o3d.geometry.TriangleMesh()
ex.vertices = o3d.utility.Vector3dVector(verts)
ex.triangles = o3d.utility.Vector3iVector(faces)

In [ ]:
o3d.visualization.draw_geometries([ex])